In [5]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')
from Utils.utils import create_URM, create_ICM, combine_matrices, create_submission, write_submission
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Evaluation.K_FoldEvaluator import K_FoldEvaluator
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
##NON-PERSONALIZED
from src.Recommenders.Base.NonPersonalizedRecommender import TopPop

#KNN
from src.Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from src.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from src.Recommenders.KNN.ItemKNN_CFCBF_Hybrid_Recommender import ItemKNN_CFCBF_Hybrid_Recommender
from src.Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender

#MATRIX-FACTORIZATION
from src.Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from src.Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from src.Recommenders.MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython, MatrixFactorization_AsySVD_Cython
from src.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender
from src.Recommenders.MatrixFactorization.IALSRecommenderLinear import IALSRecommender
from src.Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

#GRAPHIC-BASED
from src.Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender

#HYBRID
from src.Hybrids.MergingModelsByScores import MergeTwoModelsByScores, MergeThreeModelsByScores

In [6]:
##CREATION OF MATRICES AND SPLITTING IN TRAIN-TEST-VALIDATION
URM = create_URM()
ICM = create_ICM()

In [7]:
URM_stacked = combine_matrices(ICM=ICM,URM=URM)

In [8]:
rp3_recommender=RP3betaRecommender(URM_train=URM_stacked)
rp3_recommender.fit(topK=181, 
                    alpha=0.5709402717259106, 
                    beta=0.3566066452521787, 
                    implicit=True, 
                    normalize_similarity=True)

In [9]:
slime_recommender=MultiThreadSLIM_SLIMElasticNetRecommender(URM_train=URM_stacked)
slime_recommender.fit(topK=420, 
                      l1_ratio=0.06061216785568925, 
                      alpha=0.00554982938879437, 
                      workers = 20)

100%|████████████████████████████████████▉| 18032/18059 [14:00<00:00, 45.60it/s]

In [10]:
IALS_recommender=IALSRecommender(URM_train=URM_stacked)
IALS_recommender.fit(epochs=10, 
                     num_factors=42, 
                     alpha=0.7617528864750021, 
                     reg=8.926401306541349)

IALSRecommenderLinear: Epoch 1 of 10. Elapsed time 10.13 sec


100%|████████████████████████████████████▉| 18056/18059 [14:20<00:00, 45.60it/s]

IALSRecommenderLinear: Epoch 2 of 10. Elapsed time 20.37 sec
IALSRecommenderLinear: Epoch 3 of 10. Elapsed time 30.54 sec
IALSRecommenderLinear: Epoch 4 of 10. Elapsed time 40.30 sec
IALSRecommenderLinear: Epoch 5 of 10. Elapsed time 48.89 sec
IALSRecommenderLinear: Epoch 6 of 10. Elapsed time 57.30 sec
IALSRecommenderLinear: Epoch 7 of 10. Elapsed time 1.10 min
IALSRecommenderLinear: Epoch 8 of 10. Elapsed time 1.23 min
IALSRecommenderLinear: Epoch 9 of 10. Elapsed time 1.38 min
IALSRecommenderLinear: Epoch 10 of 10. Elapsed time 1.53 min
IALSRecommenderLinear: Terminating at epoch 10. Elapsed time 1.53 min


In [11]:
recommender = MergeThreeModelsByScores(URM_train=URM_stacked,recommenders=[slime_recommender,IALS_recommender,rp3_recommender])
recommender.fit(alpha=0.8257646978659972,beta=0.271877679621908,gamma=0.2321636664081369)

In [12]:
submission = create_submission(recommender)
write_submission(submission,"hybrid_KFOLD_scores")

100%|████████████████████████████████████▉| 18056/18059 [15:54<00:00, 18.91it/s]
